# Fine-tuning Spark-TTS Model


In [1]:
!pip install -q -U transformers torch torchaudio soundfile accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 150.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17

In [2]:
!pip install -q -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 52.8 MB/s eta 0:00:00


In [3]:
!pip install -q torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 88.6 MB/s eta 0:00:00


In [ ]:
!pip install -q einx

In [ ]:
# Fix the torchvision/torch version conflict
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade accelerate


## Dataset loading

In [4]:
from datasets import load_dataset

dataset = load_dataset("MBZUAI/ClArTTS")

print(f"Train samples: {len(dataset['train'])}")  # 9,500 samples
print(f"Test samples: {len(dataset['test'])}")     # 205 samples

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

data/train-00000-of-00026.parquet:   0%|          | 0.00/123M [00:00<?, ?B/s]

data/train-00001-of-00026.parquet:   0%|          | 0.00/121M [00:00<?, ?B/s]

...

data/test-00000-of-00001.parquet:   0%|          | 0.00/68.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/205 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

Train samples: 9500
Test samples: 205


In [7]:
# Load the dataset
from datasets import load_dataset
import soundfile as sf
import librosa
import os
from tqdm import tqdm

print("Loading ClArTTS...")
dataset = load_dataset("MBZUAI/ClArTTS")

print(f"Train samples: {len(dataset['train'])}")
print(f"Test samples: {len(dataset['test'])}")

# Look at one sample to understand the structure
sample = dataset['train'][0]
print("\nSample structure:")
print(f"Text: {sample['text']}")
print(f"Duration: {sample['duration']} seconds")
print(f"Sample rate: {sample['sampling_rate']} Hz")

Loading ClArTTS...


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

Train samples: 9500
Test samples: 205

Sample structure:
Text: لِأَنَّهُ لَا يَرَى أَنَّهُ عَلَى السَّفَهِ ثُمَّ مِنْ بَعْدِ ذَلِكَ حَدِيثٌ مُنْتَشِرٌ
Duration: 5.29 seconds
Sample rate: 40100 Hz


In [9]:
print(f"Audio shape: {len(sample['audio'])}")


Audio shape: 212200


In [10]:
audio = sample["audio"]
sr = sample["sampling_rate"]

print(type(audio))
print(len(audio))
print("Sampling rate:", sr)


<class 'list'>
212200
Sampling rate: 40100


## convert raw data to audio

In [11]:
import torch
import torchaudio

waveform = torch.tensor(audio).unsqueeze(0)  # [1, T]

torchaudio.save("example.wav", waveform, sr)

print("Saved example.wav")


import torchaudio
from IPython.display import Audio, display
import os

output_path = "example.wav"

if os.path.exists(output_path):
    print("Loading saved audio file...")

    # 1. Load the waveform and sample rate using torchaudio
    waveform, sample_rate = torchaudio.load(output_path)

    print(f"✓ Audio loaded successfully from: {output_path}")
    print(f"  Sample Rate: {sample_rate} Hz")
    print(f"  Duration: {waveform.shape[1] / sample_rate:.2f}s")

    # 2. Use IPython.display.Audio to create a playable widget
    # Note: waveform[0] is used to pass the single channel (mono) data
    display(Audio(waveform.numpy(), rate=sample_rate))
else:
    print(f"❌ Error: Audio file '{output_path}' not found.")

Saved example.wav
Loading saved audio file...
✓ Audio loaded successfully from: example.wav
  Sample Rate: 40100 Hz
  Duration: 5.29s


### repo cloning

In [12]:
# Clone the correct repo
!git clone https://github.com/ductran150499/SparkTTS-Finetune.git
%cd SparkTTS-Finetune

# Check what's inside
!ls -la

Cloning into 'SparkTTS-Finetune'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 144 (delta 47), reused 120 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (144/144), 10.61 MiB | 2.63 MiB/s, done.
Resolving deltas: 100% (47/47), done.
/content/SparkTTS-Finetune
total 1408
drwxr-xr-x 11 root root    4096 Dec 16 08:34 .
drwxr-xr-x  1 root root    4096 Dec 16 08:34 ..
-rw-r--r--  1 root root 1109838 Dec 16 08:34 35c44830_f000219.wav
drwxr-xr-x  2 root root    4096 Dec 16 08:34 config_axolotl
-rw-r--r--  1 root root     957 Dec 16 08:34 create_metadata.py
drwxr-xr-x  2 root root    4096 Dec 16 08:34 data_train
-rw-r--r--  1 root root     570 Dec 16 08:34 download_ljspeech.sh
-rw-r--r--  1 root root    6148 Dec 16 08:34 .DS_Store
-rw-r--r--  1 root root    2059 Dec 16 08:34 encode_audio.py
drwxr-xr-x  8 root root    4096 Dec 16 08:34 .git
drwxr-xr-x  3 root root    4096 Dec

In [13]:
!cat create_metadata.py

import os
import csv

# Đường dẫn tới folder chứa cả .wav và .txt
data_folder = "/Users/tranminhduc/Downloads/sample_data"

# File output CSV
metadata_path = os.path.join(data_folder, "metadata.csv")

# Ghi metadata.csv
with open(metadata_path, mode='w', encoding='utf-8', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["audio_path", "text"])  # Header

    for file in sorted(os.listdir(data_folder)):
        if file.endswith(".wav"):
            base_name = os.path.splitext(file)[0]
            txt_path = os.path.join(data_folder, base_name + ".txt")

            if os.path.exists(txt_path):
                with open(txt_path, "r", encoding="utf-8") as f:
                    text = f.read().strip()
                writer.writerow([file, text])
            else:
                print(f"⚠️ Không tìm thấy file .txt cho {file}")

print(f"✅ Đã tạo xong metadata.csv tại: {metadata_path}")


### Step 4: Prepare ClArTTS Data in the Required Format


In [21]:
import os
import soundfile as sf
import librosa
import numpy as np
import csv
from tqdm import tqdm

# Clear and recreate folder (to avoid duplicates)
!rm -rf /content/clartts_data
data_folder = "/content/clartts_data"
os.makedirs(data_folder, exist_ok=True)

# Process FULL dataset
num_samples = len(dataset['train'])  # All 9500 samples
print(f"Processing {num_samples} samples...")

for idx in tqdm(range(num_samples)):
    sample = dataset['train'][idx]

    # Get audio and text
    audio = np.array(sample['audio'])
    sr = sample['sampling_rate']
    text = sample['text']

    # Resample to 24kHz if needed
    if sr != 24000:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=24000)
        sr = 24000

    # Create filenames
    base_name = f"clartts_{idx:05d}"
    wav_path = os.path.join(data_folder, f"{base_name}.wav")
    txt_path = os.path.join(data_folder, f"{base_name}.txt")

    # Save audio
    sf.write(wav_path, audio, sr)

    # Save text
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write(text)

print(f"✅ Saved {num_samples} files!")



Processing 9500 samples...


100%|██████████| 9500/9500 [14:46<00:00, 10.72it/s]

✅ Saved 9500 files!


In [22]:
# Now create metadata.csv
metadata_path = os.path.join(data_folder, "metadata.csv")

with open(metadata_path, mode='w', encoding='utf-8', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["audio_path", "text"])  # Header

    for file in sorted(os.listdir(data_folder)):
        if file.endswith(".wav"):
            base_name = os.path.splitext(file)[0]
            txt_path = os.path.join(data_folder, base_name + ".txt")

            if os.path.exists(txt_path):
                with open(txt_path, "r", encoding="utf-8") as f:
                    text = f.read().strip()
                writer.writerow([file, text])

print(f"✅ Created metadata.csv at: {metadata_path}")

# Verify final structure
!ls -lh {data_folder} | head -20
!wc -l {metadata_path}

✅ Created metadata.csv at: /content/clartts_data/metadata.csv
total 1.9G
-rw-r--r-- 1 root root  163 Dec 16 08:40 clartts_00000.txt
-rw-r--r-- 1 root root 249K Dec 16 08:40 clartts_00000.wav
-rw-r--r-- 1 root root   75 Dec 16 08:40 clartts_00001.txt
-rw-r--r-- 1 root root 144K Dec 16 08:40 clartts_00001.wav
-rw-r--r-- 1 root root   92 Dec 16 08:40 clartts_00002.txt
-rw-r--r-- 1 root root 156K Dec 16 08:40 clartts_00002.wav
-rw-r--r-- 1 root root  165 Dec 16 08:40 clartts_00003.txt
-rw-r--r-- 1 root root 239K Dec 16 08:40 clartts_00003.wav
-rw-r--r-- 1 root root   84 Dec 16 08:40 clartts_00004.txt
-rw-r--r-- 1 root root 164K Dec 16 08:40 clartts_00004.wav
-rw-r--r-- 1 root root   84 Dec 16 08:40 clartts_00005.txt
-rw-r--r-- 1 root root 151K Dec 16 08:40 clartts_00005.wav
-rw-r--r-- 1 root root  142 Dec 16 08:40 clartts_00006.txt
-rw-r--r-- 1 root root 254K Dec 16 08:40 clartts_00006.wav
-rw-r--r-- 1 root root   76 Dec 16 08:40 clartts_00007.txt
-rw-r--r-- 1 root root 146K Dec 16 08:40 c

In [23]:
# Go to the repo directory
%cd /content/SparkTTS-Finetune

# Install requirements
!pip install -q -r requirement.txt

# Download pretrained Spark-TTS model
!python src/download_pretrain.py

/content/SparkTTS-Finetune
ERROR: Could not find a version that satisfies the requirement torch==2.1.2+cu121 (from versions: 2.2.0, 2.2.0+cu121, 2.2.1, 2.2.1+cu121, 2.2.2, 2.2.2+cu121, 2.3.0, 2.3.0+cu121, 2.3.1, 2.3.1+cu121, 2.4.0, 2.4.0+cu121, 2.4.1, 2.4.1+cu121, 2.5.0, 2.5.0+cu121, 2.5.1, 2.5.1+cu121, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==2.1.2+cu121
Fetching 31 files:   0% 0/31 [00:00<?, ?it/s]
special_tokens_map.json: 100% 613/613 [00:00<00:00, 5.39MB/s]

merges.txt: 1.67MB [00:00, 92.2MB/s]

config.yaml: 1.16kB [00:00, 2.76MB/s]

.gitattributes: 1.77kB [00:00, 9.83MB/s]
Fetching 31 files:   3% 1/31 [00:00<00:09,  3.10it/s]
config.json: 100% 658/658 [00:00<00:00, 5.85MB/s]

added_tokens.json: 509kB [00:00, 169MB/s]

LLM/model.safetensors:   0% 0.00/2.03G [00:00<?, ?B/s]

tokenizer_config.json: 2.58MB [00:00, 251MB/s]


README.md: 6.47kB [00:00, 20.1MB/s]


vocab.json: 0.00B [00:00, ?B/s]


vocab.json: 2.78MB [00:00, 120MB/s]


LL

In [27]:
import csv
import os

# Read the original metadata
original_metadata = "/content/clartts_data/metadata.csv"
data_folder = "/content/clartts_data"

# Read all entries
with open(original_metadata, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)  # Get header
    all_rows = list(reader)

# Take only 30% of the data (or change to 0.5 for 50%)
percentage = 0.3  # 30%
num_samples = int(len(all_rows) * percentage)
selected_rows = all_rows[:num_samples]

print(f"Original samples: {len(all_rows)}")
print(f"Selected samples: {num_samples}")

# Create new metadata file
new_metadata = os.path.join(data_folder, "metadata_subset.csv")
with open(new_metadata, 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(selected_rows)

print(f"✅ Created subset metadata: {new_metadata}")


Original samples: 9500
Selected samples: 2850
✅ Created subset metadata: /content/clartts_data/metadata_subset.csv


In [28]:
%cd /content/SparkTTS-Finetune

# Process with subset (manually specify the metadata file or copy it)
!cp /content/clartts_data/metadata_subset.csv /content/clartts_data/metadata.csv

!python -m src.process_data \
  --data_dir /content/clartts_data \
  --output_dir /content/processed_output

/content/SparkTTS-Finetune
2025-12-16 09:03:29.535166: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-16 09:03:29.553917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765875809.577351   11550 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765875809.584969   11550 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765875809.604505   11550 computation_placer.cc:177] computation placer already registered. Please check

##  Install Axolotl for Training

In [29]:
# Clone and install axolotl (LLM training framework)
%cd /content
!git clone https://github.com/OpenAccess-AI-Collective/axolotl
%cd axolotl
!pip install -e .

/content
Cloning into 'axolotl'...
remote: Enumerating objects: 52172, done.
remote: Counting objects: 100% (602/602), done.
remote: Compressing objects: 100% (388/388), done.
remote: Total 52172 (delta 421), reused 221 (delta 212), pack-reused 51570 (from 4)
Receiving objects: 100% (52172/52172), 27.06 MiB | 9.60 MiB/s, done.
Resolving deltas: 100% (37346/37346), done.
/content/axolotl
Obtaining file:///content/axolotl
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 4.3 

In [30]:
# Go back to the finetune repo
%cd /content/SparkTTS-Finetune

# Check the config file
!cat config_axolotl/full_finetune.yml

/content/SparkTTS-Finetune
base_model: /content/pretrained_models/Spark-TTS-0.5B/LLM
load_in_4bit: false
load_in_8bit: false

trust_remote_code: true
strict: false

datasets:
  - path: /content/SparkTTS-Finetune/sample_output/sample_data.jsonl
    type: completion

dataset_prepared_path:
val_set_size: 0.05
output_dir: /content/drive/MyDrive/SparkTTS-model


sequence_len: 1024
sample_packing: false
eval_sample_packing: false
pad_to_sequence_len: true

wandb_project:
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:

gradient_accumulation_steps: 8
micro_batch_size: 1
num_epochs: 50
optimizer: adamw_torch_fused
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: true
fp16: false
tf32: false

gradient_checkpointing: true
gradient_checkpointing_kwargs:
  use_reentrant: false

early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 50
xformers_attention:
flash_attention: false

warmup_steps: 10
evals_per_epoch: 1
save_step

In [9]:
# Update config with correct path
config_content = """base_model: /content/SparkTTS-Finetune/pretrained_models/Spark-TTS-0.5B/LLM
load_in_4bit: false
load_in_8bit: false

trust_remote_code: true
strict: false

datasets:
  - path: /content/processed_output/clartts_data.jsonl
    type: completion

dataset_prepared_path:
val_set_size: 0.05
output_dir: /content/finetuned_model

sequence_len: 1024
sample_packing: false
eval_sample_packing: false
pad_to_sequence_len: true

wandb_project:
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:

gradient_accumulation_steps: 8
micro_batch_size: 1
num_epochs: 20
optimizer: adamw_torch_fused
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: true
fp16: false
tf32: false

gradient_checkpointing: true
gradient_checkpointing_kwargs:
  use_reentrant: false

early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 50
xformers_attention:
flash_attention: false

warmup_steps: 10
evals_per_epoch: 1
save_steps: 200
debug:
deepspeed:
weight_decay: 0.0
"""

with open('config_axolotl/full_finetune.yml', 'w') as f:
    f.write(config_content)

print("✅ Config fixed with correct path!")

✅ Config fixed with correct path!


In [11]:
!sed -i 's/num_epochs: 20/num_epochs: 3/g' config_axolotl/full_finetune.yml

In [12]:
!accelerate launch -m axolotl.cli.train config_axolotl/full_finetune.yml

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-12-16 09:55:30.651223: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-16 09:55:30.672783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765878930.696285   25550 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register 

# Create complete model package

In [50]:
import shutil
import os
from huggingface_hub import HfApi

print("📦 Creating complete model package...")

# 1. Copy base Spark-TTS structure
complete_model_dir = "/content/Spark-TTS-Arabic-Complete"
shutil.copytree(
    "pretrained_models/Spark-TTS-0.5B",
    complete_model_dir,
    dirs_exist_ok=True
)
print("✅ Copied base structure")

# 2. Replace LLM with our fine-tuned version
print("🔄 Replacing LLM with our fine-tuned version...")
shutil.rmtree(f"{complete_model_dir}/LLM")
shutil.copytree("/content/finetuned_model", f"{complete_model_dir}/LLM")
print("✅ LLM replaced!")

# 3. Create a README
readme = """
Ur readme file here
"""

with open(f"{complete_model_dir}/README.md", "w", encoding="utf-8") as f:
    f.write(readme)
print("✅ README created!")

# 4. Upload to HuggingFace
print("\n🚀 Uploading to HuggingFace...")
api = HfApi()
repo_id = "azeddinShr/Spark-TTS-Arabic-Complete"

try:
    api.create_repo(repo_id=repo_id, exist_ok=True)
    print(f"✅ Repository created: {repo_id}")
except Exception as e:
    print(f"Repository exists: {e}")

api.upload_folder(
    folder_path=complete_model_dir,
    repo_id=repo_id,
    repo_type="model",
    commit_message="Complete Spark-TTS with Arabic fine-tuned LLM"
)

print(f"\n🎉 Model uploaded!")
print(f"🔗 https://huggingface.co/{repo_id}")

📦 Creating complete model package...
✅ Copied base structure
🔄 Replacing LLM with our fine-tuned version...
✅ LLM replaced!
✅ README created!

🚀 Uploading to HuggingFace...
✅ Repository created: azeddinShr/Spark-TTS-Arabic-Complete


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...eckpoint-600/scheduler.pt: 100%|##########| 1.47kB / 1.47kB            

  ...nt-1000/training_args.bin: 100%|##########| 7.31kB / 7.31kB            

...

  ...kpoint-600/tokenizer.json: 100%|##########| 14.1MB / 14.1MB            

  ...point-1000/tokenizer.json: 100%|##########| 14.1MB / 14.1MB            

  ...ete/LLM/model.safetensors:   1%|          | 8.37MB / 1.31GB            

  ...int-800/model.safetensors:   2%|1         | 25.1MB / 1.31GB            


🎉 Model uploaded!
🔗 https://huggingface.co/azeddinShr/Spark-TTS-Arabic-Complete


# Model Inference

In [52]:
print("📦 Installing dependencies...")
# 2. Clone Spark-TTS repo
print("📥 Cloning Spark-TTS...")
!git clone https://github.com/SparkAudio/Spark-TTS
%cd Spark-TTS

📦 Installing dependencies...
📥 Cloning Spark-TTS...
Cloning into 'Spark-TTS'...
remote: Enumerating objects: 384, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 384 (delta 120), reused 99 (delta 99), pack-reused 213 (from 1)
Receiving objects: 100% (384/384), 7.07 MiB | 5.19 MiB/s, done.
Resolving deltas: 100% (156/156), done.
/content/SparkTTS-Finetune/Spark-TTS/Spark-TTS


In [53]:

# 4. Setup paths
import sys
sys.path.insert(0, '/content/Spark-TTS')
sys.path.insert(0, '/content/Spark-TTS/cli')

from SparkTTS import SparkTTS
from transformers import AutoModelForCausalLM
import torch
import soundfile as sf
from IPython.display import Audio, display

In [54]:

# ===== CUSTOMIZE HERE =====
REFERENCE_AUDIO_PATH = "/content/Spark-TTS/reference.wav"
REFERENCE_TEXT = "لَا يَمُرُّ يَوْمٌ إِلَّا وَأَسْتَقْبِلُ عِدَّةَ رَسَائِلَ."
TEST_TEXT = """إِنَّ العِلْمَ نُورٌ يُنِيرُ العُقُولَ، وَيَرْفَعُ قَدْرَ الإِنْسَانِ.
بِالعِلْمِ نَفْهَمُ العَالَمَ مِنْ حَوْلِنَا، وَنَبْنِي مُسْتَقْبَلًا أَفْضَلَ.
وَمَنْ سَلَكَ طَرِيقَ العِلْمِ، وَجَدَ فِيهِ أَمَلًا وَنَجَاحًا."""
# ==========================


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Using device: {device}")

# 5. Load base model
print("\n🔄 Loading base SparkTTS model...")
tts = SparkTTS("pretrained_models/Spark-TTS-0.5B", device)
print("✅ Base model loaded!")

# 6. Test baseline (before fine-tuning)
print("\n🎤 Generating with BASELINE model...")
baseline_wav = tts.inference(
    TEST_TEXT,
    REFERENCE_AUDIO_PATH,
    prompt_text=REFERENCE_TEXT
)
sf.write("baseline_output.wav", baseline_wav, samplerate=16000)
print("✅ Baseline generated!")
print("\n🔊 Baseline output:")
display(Audio("baseline_output.wav"))

🖥️ Using device: cuda

🔄 Loading base SparkTTS model...


/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Missing tensor: mel_transformer.spectrogram.window
Missing tensor: mel_transformer.mel_scale.fb
✅ Base model loaded!

🎤 Generating with BASELINE model...


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


✅ Baseline generated!

🔊 Baseline output:


In [ ]:
# Download Ibrahim's full model
from huggingface_hub import snapshot_download

dir = "azeddin_model"
snapshot_download(
    repo_id="azeddinShr/Spark-TTS-Arabic-Complete",
    local_dir=dir,
    local_dir_use_symlinks=False
)

# Try to load it (might fail - different structure)
ibrahim_tts = SparkTTS(dir, device)
ibrahim_wav = ibrahim_tts.inference(TEST_TEXT, '/content/example.wav', prompt_text=REFERENCE_TEXT)
sf.write("ibrahim.wav", ibrahim_wav, samplerate=16000)
display(Audio("ibrahim.wav"))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 95 files:   0%|          | 0/95 [00:00<?, ?it/s]

BiCodec/model.safetensors:   0%|          | 0.00/626M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.yaml: 0.00B [00:00, ?B/s]

...

README.md: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

wav2vec2-large-xlsr-53/pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

Missing tensor: mel_transformer.spectrogram.window
Missing tensor: mel_transformer.mel_scale.fb
